In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score as r2
from urllib import request
import json

In [2]:
X_test = pd.read_csv("./output/X_test.csv")
y_test = pd.read_csv("./output/y_test.csv")

X_test.head(3)

,REF,Company (Manufacturer),Company Location,Review Date,Country of Bean Origin,Specific Bean Origin or Bar Name,Cocoa Percent,Ingredients,Most Memorable Characteristics,Rating
0,607,Soma,Canada,2010,Blend,"Black Science Blend 1, DR, Ecuador, Peru",70%,"3- B,S,C","complex, nut, dried fruit, earth",3.75
1,2362,Mellow,Denmark,2019,Dominican Republic,"Oko Caribe, El Cibao, 2018 Harvest",70%,"2- B,S*","dry, floral, dirty, coconut",2.75
2,141,Pierre Marcolini,Belgium,2007,Madagascar,"Sambirano, Ambanja, Madagascar",72%,"5- B,S,C,V,L","tangy, floral, spicy, cocoa",4.00


In [3]:
def get_prediction(manufacturer, company_location, review_date, bean_origin_country, 
                   bean_origin_specific, cocoa_percent, ingredients, mem_characteristics):
    
    body = {"Company (Manufacturer)": manufacturer, 
            "Company Location": company_location, 
            "Review Date": review_date,
            "Country of Bean Origin": bean_origin_country, 
            "Specific Bean Origin or Bar Name": bean_origin_specific,
            "Cocoa Percent": cocoa_percent,
            "Ingredients": ingredients, 
            "Most Memorable Characteristics": mem_characteristics}

    myurl = "http://localhost:8180/predict"
    req = request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = request.urlopen(req, jsondataasbytes)
    res = json.loads(response.read())
    if res.get('predictions', None) is not None:
        return res['predictions']
    else:
        return 0

In [4]:
%%time
sample_count = 50
predictions = X_test[['Company (Manufacturer)', 'Company Location', 'Review Date',
                      'Country of Bean Origin', 'Specific Bean Origin or Bar Name',
                      'Cocoa Percent', 'Ingredients', 'Most Memorable Characteristics']].\
              iloc[:sample_count].apply(lambda x: get_prediction(x[0], x[1], x[2],
                                                      x[3], x[4],
                                                      x[5], x[6], x[7]), 1)

Wall time: 4.97 s


In [5]:
print(f"Train R2:\t {r2(y_test[:sample_count], predictions):.3}\n")

Train R2:	 0.181

